In [1]:
# ! pip install agent-framework-azure-ai -U

In [2]:
import os

from azure.identity.aio import AzureCliCredential
from azure.ai.projects.aio import AIProjectClient
from dotenv import load_dotenv

from azure.ai.agents.models import FilePurpose,FileSearchTool
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent,HostedFileSearchTool,HostedVectorStoreContent

In [3]:
load_dotenv()

True

In [4]:
async def create_vector_store(client: AIProjectClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = '../../files/demo.md'
    file = await client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

In [5]:
async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):

        file_id, vector_store = await create_vector_store(client)
        file_search = FileSearchTool(vector_store_ids=[vector_store.vector_store_id])
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="PythonRAGAgent",
            instructions="""
                    You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                    - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                    "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                    - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                    - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                    - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                    - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                    You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                    """,
            tools = file_search.definitions,
            tool_resources= file_search.resources
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)


        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
            ) as agent:
                    

            print("Agent created. You can now ask questions about the uploaded document.")

            query = "What's GraphRAG?"

            async for chunk in agent.run_stream(query, tools=HostedFileSearchTool(inputs=vector_store)):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

Uploaded file, file ID: assistant-KTUZiULQcLW8KZw5dCUKBy
Created vector store, ID: vs_n0I71Ytm0LgM4sfYPilDh2GH
Agent created. You can now ask questions about the uploaded document.
GraphRAG is an AI-based content interpretation and search capability. It uses large language models (LLMs) to parse data and create a knowledge graph, which it then uses to answer user questions about a user-provided private dataset.

GraphRAG can connect information across large volumes of data and use those connections to answer complex questions that are difficult or impossible to answer with traditional keyword or vector-based searches. It can handle questions that span many documents and even thematic questions, such as identifying top themes in a dataset.

The intended use of GraphRAG includes supporting critical information discovery and analysis, especially where information spans many documents, is noisy or mixed with misinformation, or where questions are abstract or thematic. It is designed for us